# Memory
Open AI 和 Azure OpenAI的GPT模型没有记忆, 所的信息和输入要以上下文形式加人文本 Prompt， 通过API发送给AI模型. 
Semantic Kernel提供外部记忆来帮助存储和检索信息。

## Embedding and Semantic Memory Search

Embedding是将单词或其他数据表示为高维空间中的向量的一种方法。向量就像具有方向和长度的箭头。高维意味着空间具有许多维度，超过我们能看到或想象的范围。其核心思想是相似的单词或数据将具有相似的向量，而不同的单词或数据将具有不同的向量。这有助于我们测量它们之间的关系或无关性，并执行各种操作，如加法、减法、乘法等。嵌入对于人工智能模型非常有用，因为它们可以以计算机能够理解和处理的方式捕捉单词或数据的含义和上下文。

基本上，你需要取一个句子、段落或整个文本页面，然后生成相应的嵌入向量。

当进行查询时(Semantic Search)，将查询转换为其嵌入表示，然后通过所有现有的嵌入向量进行搜索，找到最相似的那些。这类似于在Bing上进行搜索查询，它会提供与您的查询接近的多个结果。语义记忆不太可能给您一个完全匹配的结果，但它总会给您一组按照您的查询与其他文本段相似程度排名的匹配结果。

Semantic Kernel 提供了非常简洁易用的Interface 为我们完成任务并隐藏了细节。

1: MemoryBuilder
在 Semantic Kernel V1 （beta) 中， Memory 的配置已从 Kernel中独立出来
我们现在创建一个Memory 对象，并配置内存作为记忆体。

In [1]:
// 安装 nuget 包
#r "nuget: Microsoft.SemanticKernel, 1.0.0-beta8"
#r "nuget: System.Linq.Async, 6.0.1"
#r "nuget: Microsoft.Extensions.Logging.Console, 6.0.0.0"


// 如遇到这个错误请忽略并继续。 Error: Microsoft.Extensions.Logging.Console version 6.0.0.0 cannot be added because version 6.0.0 was added previously

Installed Packages Microsoft.Extensions.Logging.Console, 6.0.0 Microsoft.SemanticKernel, 1.0.0-beta8 System.Linq.Async, 6.0.1

In [2]:
using Microsoft.SemanticKernel.Plugins.Memory;
using Microsoft.SemanticKernel.Connectors.AI.OpenAI;

var model = "text-embedding-ada-002";
var apiKey = Environment.GetEnvironmentVariable("OPENAI__APIKEY");
ArgumentNullException.ThrowIfNull(apiKey);

var memoryBuilder = new MemoryBuilder();
memoryBuilder.WithOpenAITextEmbeddingGenerationService(model, apiKey);

memoryBuilder.WithMemoryStore(new VolatileMemoryStore());

var memory = memoryBuilder.Build();

在文件`fun_fact_about_contries.text`中包含了160多个国家的有趣信息, 让我们把它存入memory

In [15]:
using System.IO;
string[] lines = File.ReadAllLines("fun_fact_about_contries.text");

var  i = 0;
foreach (string line in lines) {

    if (!String.IsNullOrEmpty(line)) {
        await memory.SaveInformationAsync("FunFactAboutContries", line, $"id_{i++}" , line);
    }
}

现在我们就可以向这个文件提问题了。

In [21]:
var q1 = "which country makes most toys in the world";
var q2 = "哪个国家拥有最多的湖泊";

var ask = async (string question) => {
    var facts = memory.SearchAsync("FunFactAboutContries", question,  limit: 1, minRelevanceScore: 0.7);

    await foreach (var fact in facts)
    {
        Console.WriteLine("  Question   : " + question);
        Console.WriteLine("  Fact   : " + fact.Metadata.Description);
        Console.WriteLine("  Relevance: " + fact.Relevance);
        Console.WriteLine("");
    }
};

await ask(q1);
await ask(q2);


  Question   : which country makes most toys in the world
  Fact   : China manufactures 70% of the world toys.
  Relevance: 0.8835527300834656

  Question   : 哪个国家拥有最多的湖泊
  Fact   : Canada has the most lakes on the planet. There are more lakes in Canada than in every other country in the world put together.
  Relevance: 0.7925511598587036



以上我们实际只需要用到 `ISemanticTextMemory`提供的`SaveInformationAsync`与`SearchAsync` 两个方法。 Semantic Kernel 将 向量生成，存储 和 查询 全部实现都隐藏在`SemanticTextMemory`实现中了。

## 实践：修改上一个lab 中的 JokeFun, 从以上memory中找到关于一个国家的信息然后再生成一个 joke

In [23]:

// 安装 nuget 包
#r "nuget: Microsoft.SemanticKernel, 1.0.0-beta8"
#r "nuget: Microsoft.Extensions.Logging.Console, 6.0.0.0"

// 如遇到这个错误请忽略并继续。 Error: Microsoft.Extensions.Logging.Console version 6.0.0.0 cannot be added because version 6.0.0 was added previously

Error: (1,1): error CS1024: Preprocessor directive expected

In [24]:
using Microsoft.Extensions.Logging;
using Microsoft.SemanticKernel;

var builder = new KernelBuilder();

var model = "gpt-3.5-turbo-1106";
var apiKey = Environment.GetEnvironmentVariable("OPENAI__APIKEY");

ArgumentNullException.ThrowIfNull(apiKey);

var loggerFactory = LoggerFactory.Create(builder => { 
    builder.SetMinimumLevel(LogLevel.Information);  
    builder.AddConsole(); });

var kernel = builder
.WithLoggerFactory(loggerFactory)
.WithOpenAIChatCompletionService(model, apiKey)
.Build();



In [30]:
using Microsoft.SemanticKernel.Orchestration;
const string JokeFuncDefinition = "请基于以下信息 {{ $fact }}，生成关与{{ $input}} 的段子, 要求用黑色幽默";

var jokeFunc = kernel.CreateSemanticFunction(JokeFuncDefinition);

var question = "24 小时流淌红酒的喷泉的国家";

var facts = memory.SearchAsync("FunFactAboutContries", question,  limit: 1, minRelevanceScore: 0.7);

await foreach (var fact in facts)
{
    var result = await kernel.RunAsync(jokeFunc, new ContextVariables() {
    ["input"] = question,
    ["fact"] = fact.Metadata.Description
});
    Console.WriteLine(result.GetValue<string>());

}


在意大利，有一个喷泉24小时不间断地流淌着红酒。这让人想起一句话：如果你觉得生活像一杯红酒，那么在意大利，生活就像一个24小时不停止的自助餐厅。
